In [121]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [122]:
weather_df = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/weather_data/DWD_ICON-EU.csv')
solar_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_total_production.csv')

In [123]:
weather_df.sort_values(by='ref_datetime', inplace=True)

In [124]:
weather_df = weather_df.groupby(["valid_datetime","latitude","longitude"]).last().reset_index()

In [125]:
print(weather_df.columns)

Index(['valid_datetime', 'latitude', 'longitude', 'ref_datetime',
       'Temperature', 'WindSpeed', 'WindSpeed:100', 'WindDirection:100',
       'CloudCover', 'RelativeHumidity', 'PressureReducedMSL',
       'SolarDownwardRadiation', 'TotalPrecipitation'],
      dtype='object')


In [126]:
weather_df.reset_index(inplace=True)

In [127]:
def resample_and_interpolate(group):
    return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()

In [128]:
weather_df.valid_datetime = pd.to_datetime(weather_df.valid_datetime)
weather_df = weather_df.set_index(["valid_datetime","latitude","longitude"])
df_resampled = weather_df.groupby(['latitude', 'longitude'], group_keys=False).apply(resample_and_interpolate)

C:\Users\paulh\AppData\Local\Temp\ipykernel_17340\2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
C:\Users\paulh\AppData\Local\Temp\ipykernel_17340\2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
C:\Users\paulh\AppData\Local\Temp\ipykernel_17340\2834871732.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()
C:\Users\paulh\AppData\Local\Temp\ipykernel_17340

In [129]:
df_resampled = df_resampled.reset_index()

In [130]:
solar_total.timestamp_utc = pd.to_datetime(solar_total.timestamp_utc)

In [131]:
df_resampled.drop(columns=['index','ref_datetime'], inplace=True)

In [132]:
# df_resampled = df_resampled.groupby("valid_datetime").mean().reset_index()

In [143]:
df_resampled_merged = pd.merge(df_resampled, solar_total, how='left', left_on='valid_datetime', right_on='timestamp_utc')

In [144]:
df_resampled_merged.columns

Index(['valid_datetime', 'latitude', 'longitude', 'Temperature', 'WindSpeed',
       'WindSpeed:100', 'WindDirection:100', 'CloudCover', 'RelativeHumidity',
       'PressureReducedMSL', 'SolarDownwardRadiation', 'TotalPrecipitation',
       'timestamp_utc', 'generation_mw', 'installed_capacity_mwp',
       'capacity_mwp'],
      dtype='object')

solar

In [145]:
df_resampled_merged_solar = df_resampled_merged.loc[~(df_resampled_merged.latitude == 53.935) & ~(df_resampled_merged.longitude == 1.8645)]

In [146]:
df_resampled_merged_solar = df_resampled_merged_solar.groupby("valid_datetime").mean().reset_index()

In [147]:
df_resampled_merged_solar.columns

Index(['valid_datetime', 'latitude', 'longitude', 'Temperature', 'WindSpeed',
       'WindSpeed:100', 'WindDirection:100', 'CloudCover', 'RelativeHumidity',
       'PressureReducedMSL', 'SolarDownwardRadiation', 'TotalPrecipitation',
       'timestamp_utc', 'generation_mw', 'installed_capacity_mwp',
       'capacity_mwp'],
      dtype='object')

In [148]:
def set_up_solar_features(df):
    df["hour"] = df.valid_datetime.dt.hour
    df["day_of_year"] = df.valid_datetime.dt.dayofyear
    df["cos_day_of_year"] = np.cos(2 * np.pi * df.day_of_year / 365)
    df["cos_hour"] = np.cos(2 * np.pi * df.hour / 24)
    df["Mean_SolarDownwardRadiation"] = df.SolarDownwardRadiation
    df["Mean_Temperature"] = df.Temperature
    df["Std_Temperature"] = df.Temperature.std()
    df["SolarDownwardRadiation_RW_Mean_30min"] = df.Mean_SolarDownwardRadiation.rolling(window=1, min_periods=1).mean()
    df["SolarDownwardRadiation_RW_Mean_1hour"] = df.Mean_SolarDownwardRadiation.rolling(window=2, min_periods=1).mean()
    df["SolarDownwardRadiation_dwd_Mean_Lag_30min"] = df.Mean_SolarDownwardRadiation.shift(1)
    df["SolarDownwardRadiation_dwd_Mean_Lag_1h"] = df.Mean_SolarDownwardRadiation.shift(2)
    df["SolarDownwardRadiation_dwd_Mean_Lag_24h"] = df.Mean_SolarDownwardRadiation.shift(48)
    return df

In [149]:
df_resampled_merged_solar1 = set_up_solar_features(df_resampled_merged_solar)

In [150]:
def pv_temperature_efficiency(irradiance, ambient_temp, NOCT=45, wind_speed=1, eta_0=0.18, beta=0.004):
    # Calculate cell temperature using the simplified NOCT model
    Tc = ambient_temp + (NOCT - 20) * (irradiance / 800)
    
    # Calculate the efficiency loss due to increased cell temperature
    efficiency = eta_0 * (1 - beta * (Tc - 25))
    
    return Tc, efficiency

In [151]:
df_resampled_merged_solar1["Panel_Temperature_dwd_mean"], df_resampled_merged_solar1["Panel_Efficiency_dwd_mean"] = pv_temperature_efficiency(df_resampled_merged_solar1["Mean_SolarDownwardRadiation"], df_resampled_merged_solar1["Mean_Temperature"])

In [152]:
df_resampled_merged_solar1["Panel_Temperature_dwd_std"] = df_resampled_merged_solar1.Panel_Temperature_dwd_mean.std()
df_resampled_merged_solar1["Panel_Efficiency_dwd_std"] = df_resampled_merged_solar1.Panel_Efficiency_dwd_mean.std()

In [153]:
df_resampled_merged_solar1

,valid_datetime,latitude,longitude,Temperature,WindSpeed,WindSpeed:100,WindDirection:100,CloudCover,RelativeHumidity,PressureReducedMSL,...,Std_Temperature,SolarDownwardRadiation_RW_Mean_30min,SolarDownwardRadiation_RW_Mean_1hour,SolarDownwardRadiation_dwd_Mean_Lag_30min,SolarDownwardRadiation_dwd_Mean_Lag_1h,SolarDownwardRadiation_dwd_Mean_Lag_24h,Panel_Temperature_dwd_mean,Panel_Efficiency_dwd_mean,Panel_Temperature_dwd_std,Panel_Efficiency_dwd_std
0,2024-10-07 00:00:00+00:00,52.381681,0.650333,14.127000,4.818000,8.369000,226.312500,0.89300,93.613000,99466.75350,...,3.752767,0.000,0.000,NaN,NaN,NaN,14.127000,0.187829,5.811561,0.004184
1,2024-10-07 00:30:00+00:00,52.381681,0.650333,13.962250,4.939500,8.518500,232.540250,0.89825,92.788750,99514.00425,...,3.752767,0.000,0.000,0.000,NaN,NaN,13.962250,0.187947,5.811561,0.004184
2,2024-10-07 01:00:00+00:00,52.381681,0.650333,13.797500,5.061000,8.668000,238.768000,0.90350,91.964500,99561.25500,...,3.752767,0.000,0.000,0.000,0.000,NaN,13.797500,0.188066,5.811561,0.004184
3,2024-10-07 01:30:00+00:00,52.381681,0.650333,13.537000,5.072250,8.706500,239.918500,0.86175,91.315500,99557.52450,...,3.752767,0.000,0.000,0.000,0.000,NaN,13.537000,0.188253,5.811561,0.004184
4,2024-10-07 02:00:00+00:00,52.381681,0.650333,13.276500,5.083500,8.745000,241.069000,0.82000,90.666500,99553.79400,...,3.752767,0.000,0.000,0.000,0.000,NaN,13.276500,0.188441,5.811561,0.004184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,2024-10-13 04:00:00+00:00,52.381681,0.650333,6.834667,1.309167,3.917667,286.038833,0.94900,92.535333,101830.69250,...,3.752767,1.617,1.617,1.617,1.617,22.183167,6.885198,0.193043,5.811561,0.004184
297,2024-10-13 04:30:00+00:00,52.381681,0.650333,6.685750,1.295250,3.830250,287.283750,0.96175,92.710750,101846.30475,...,3.752767,1.617,1.617,1.617,1.617,32.452500,6.736281,0.193150,5.811561,0.004184
298,2024-10-13 05:00:00+00:00,52.381681,0.650333,6.536833,1.281333,3.742833,288.528667,0.97450,92.886167,101861.91700,...,3.752767,1.617,1.617,1.617,1.617,42.721833,6.587365,0.193257,5.811561,0.004184
299,2024-10-13 05:30:00+00:00,52.381681,0.650333,6.387917,1.267417,3.655417,289.773583,0.98725,93.061583,101877.52925,...,3.752767,1.617,1.617,1.617,1.617,52.991167,6.438448,0.193364,5.811561,0.004184


In [156]:
df_resampled_merged_solar1.generation_mw.to_clipboard()

In [160]:
df_resampled_merged_solar1.generation_mw.shift(periods= 48).to_clipboard()

In [161]:
df_resampled_merged_solar1["solar_mw_lag_24h"] = df_resampled_merged_solar1.generation_mw.shift(periods= 48)

In [162]:
df_resampled_merged_solar2 = df_resampled_merged_solar1[[ 
    "Mean_SolarDownwardRadiation",
    "SolarDownwardRadiation_RW_Mean_30min",
    "SolarDownwardRadiation_RW_Mean_1hour",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_1h",
    "SolarDownwardRadiation_dwd_Mean_Lag_24h",
    "Panel_Efficiency_dwd_mean",
    "Panel_Efficiency_dwd_std",
    "Panel_Temperature_dwd_mean",
    "Panel_Temperature_dwd_std",
    "Std_Temperature",
    "Mean_Temperature",
    "cos_hour",
    "cos_day_of_year","solar_mw_lag_24h","generation_mw"]]

In [163]:
df_resampled_merged_solar2.dropna(inplace=True)

C:\Users\paulh\AppData\Local\Temp\ipykernel_17340\3763770473.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resampled_merged_solar2.dropna(inplace=True)


In [164]:
df_resampled_merged_solar2

,Mean_SolarDownwardRadiation,SolarDownwardRadiation_RW_Mean_30min,SolarDownwardRadiation_RW_Mean_1hour,SolarDownwardRadiation_dwd_Mean_Lag_30min,SolarDownwardRadiation_dwd_Mean_Lag_1h,SolarDownwardRadiation_dwd_Mean_Lag_24h,Panel_Efficiency_dwd_mean,Panel_Efficiency_dwd_std,Panel_Temperature_dwd_mean,Panel_Temperature_dwd_std,Std_Temperature,Mean_Temperature,cos_hour,cos_day_of_year,solar_mw_lag_24h,generation_mw
48,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188265,0.004184,13.520500,5.811561,3.752767,13.52050,1.000000e+00,0.14154,0.000000,0.000000
49,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188252,0.004184,13.539250,5.811561,3.752767,13.53925,1.000000e+00,0.14154,0.000000,0.000000
50,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188238,0.004184,13.558000,5.811561,3.752767,13.55800,9.659258e-01,0.14154,0.000000,0.000000
51,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188266,0.004184,13.519000,5.811561,3.752767,13.51900,9.659258e-01,0.14154,0.000000,0.000000
52,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188294,0.004184,13.480000,5.811561,3.752767,13.48000,8.660254e-01,0.14154,0.000000,0.000000
53,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188339,0.004184,13.418500,5.811561,3.752767,13.41850,8.660254e-01,0.14154,0.000000,0.000000
54,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188383,0.004184,13.357000,5.811561,3.752767,13.35700,7.071068e-01,0.14154,0.000000,0.000000
55,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188406,0.004184,13.325250,5.811561,3.752767,13.32525,7.071068e-01,0.14154,0.000000,0.000000
56,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188429,0.004184,13.293500,5.811561,3.752767,13.29350,5.000000e-01,0.14154,0.000000,0.000000
57,0.0000,0.0000,0.00000,0.0000,0.0000,0.00000,0.188497,0.004184,13.198750,5.811561,3.752767,13.19875,5.000000e-01,0.14154,0.000000,0.000000
